## ETAPA 4: MODELOS DE MACHINE LEARNING CON LOS AGREGADOS DE "SEX" Y "AGE GROUP"

Este notebook incluye los datos de agregados "Both" y "All" de las categorías "Sex" y "Age group", respectivamente.

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# modelos lineales
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import HuberRegressor 
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV

# modelos de arboles
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
#!pip install xgboost
#pip install pydot
from xgboost import XGBRegressor
#!pip install mapie
from mapie.regression import MapieRegressor

# otros
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import PowerTransformer

import pickle
import joblib


### *Preparar Datos*

In [2]:
# Importar el dataset
df = pd.read_csv("../13 - Exports (preprocesamiento)/inmigrantes_merge.csv")

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9360 entries, 0 to 9359
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Year                               9360 non-null   int64  
 1   Nationality code                   9360 non-null   object 
 2   Sex                                9360 non-null   object 
 3   Age group                          9360 non-null   object 
 4   Immigrant count                    9360 non-null   int64  
 5   Unemployment %                     9360 non-null   float64
 6   Political and Violence Percentile  9360 non-null   float64
 7   Probability of dying young         9360 non-null   float64
 8   Rule of Law Percentile             9360 non-null   float64
 9   Salaried workers %                 9360 non-null   float64
 10  GDP_growth                         9360 non-null   float64
 11  Inflation_annual                   9360 non-null   float

,Year,Nationality code,Sex,Age group,Immigrant count,Unemployment %,Political and Violence Percentile,Probability of dying young,Rule of Law Percentile,Salaried workers %,...,Non-state_deaths,Intrastate_deaths,Interstate_deaths,Number of residents,Political regime,Homicide Rate,Number of Turist,Spanish language,Restricciones_pandemia,Año post_pandemia
0,2008,DZA,Both,0 - 14,759,11.33,14.90,3.7,24.52,67.41,...,0,345,0,51922,3,0.95,44400000,0,0,0
1,2008,PER,Males,35 - 44,2938,4.03,17.31,5.1,25.96,44.47,...,0,40,0,60185,7,5.27,44400000,1,0,0
2,2008,PER,Males,45 - 54,1128,4.03,17.31,5.1,25.96,44.47,...,0,40,0,60185,7,5.27,44400000,1,0,0
3,2008,PER,Males,55 - 64,265,4.03,17.31,5.1,25.96,44.47,...,0,40,0,60185,7,5.27,44400000,1,0,0
4,2008,PER,Males,65+,156,4.03,17.31,5.1,25.96,44.47,...,0,40,0,60185,7,5.27,44400000,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9355,2022,PAK,Males,55 - 64,330,5.60,6.60,5.8,25.00,42.14,...,0,670,0,68821,6,4.21,59310000,0,0,1
9356,2022,PAK,Females,55 - 64,146,5.60,6.60,5.8,25.00,42.14,...,0,670,0,31675,6,4.21,59310000,0,0,1
9357,2022,PAK,Both,65+,169,5.60,6.60,5.8,25.00,42.14,...,0,670,0,100496,6,4.21,59310000,0,0,1
9358,2022,PAK,Males,65+,99,5.60,6.60,5.8,25.00,42.14,...,0,670,0,68821,6,4.21,59310000,0,0,1


Primero, evaluremos modelos que no aceptan datos nulos y, posteriormente los modelos de árboles que sí los aceptan. Luego compararemos las distintas métricas juntándolas en dataframes para una mejor comparación.

### *Modelos Que No Aceptan Datos Nulos*

Antes de proceder, debemos remover a Senegal de la lista de de paises ya que tenemos datos nulos para la variable Tasa de Homidicidios, de manera que no haya conflicto con los modelos que evaluaremos. Además, haremos de la varaible "Year" una variable ordinal y el resto de variables categóricas a variables dummy (los regímenes políticos ya están en formato ordinal).

En el caso de "Year", simplemente restaremos 2007 a la columna entera, y para el resto de las variables objeto usaremos la funcion *.get_dummies()*.

In [115]:
# hacer copia del df removiendo Senegal que presenta datos nulos para tasa de homicidios
df_nonull = df[df['Nationality code'] != 'SEN'].copy()

# Transformar Year a variable ordinal de 1 (2008) a 15 (2022)
df_nonull['Year'] = df_nonull['Year'] - 2007

# Generar variables dummies a partir de nuestras variables categóricas "object" (no ordinales)
df_nonull = pd.get_dummies(df_nonull)

# Convertir las variables dummies booleanas en "int"
col_bool = df_nonull.select_dtypes(include = ['bool']).columns
df_nonull[col_bool] = df_nonull[col_bool].astype(int)

# Verificar cambio
df_nonull.info()
df_nonull

<class 'pandas.core.frame.DataFrame'>
Index: 9000 entries, 0 to 9359
Data columns (total 70 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Year                                      9000 non-null   int64  
 1   Immigrant count                           9000 non-null   int64  
 2   Unemployment %                            9000 non-null   float64
 3   Political and Violence Percentile         9000 non-null   float64
 4   Probability of dying young                9000 non-null   float64
 5   Rule of Law Percentile                    9000 non-null   float64
 6   Salaried workers %                        9000 non-null   float64
 7   GDP_growth                                9000 non-null   float64
 8   Inflation_annual                          9000 non-null   float64
 9   Liberal democracy index                   9000 non-null   float64
 10  Health equality                          

,Year,Immigrant count,Unemployment %,Political and Violence Percentile,Probability of dying young,Rule of Law Percentile,Salaried workers %,GDP_growth,Inflation_annual,Liberal democracy index,...,Continent_America,Continent_Asia,Continent_Europe,Sub-region_Africa,Sub-region_Asia,Sub-region_Central America and Caribbean,Sub-region_European Union,Sub-region_North America,Sub-region_Rest of Europe,Sub-region_South America
0,1,759,11.33,14.90,3.7,24.52,67.41,2.40,15.31,0.164,...,0,0,0,1,0,0,0,0,0,0
1,1,2938,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
2,1,1128,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
3,1,265,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
4,1,156,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9355,15,330,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0
9356,15,146,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0
9357,15,169,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0
9358,15,99,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0


Separemos el conjunto train/test y escalemos los datos.

In [116]:
# Separar variables input y variable target "Immigrant count" de df_null (dataframe sin atos nulos)
X = df_nonull.drop("Immigrant count", axis = 1) # variables predictoras
y = df_nonull["Immigrant count"]  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 58) # separar datos en conjunto train y test en un 75% / 25%
scaler_nonull = MinMaxScaler() # definir scaler de datos 
X_train = scaler_nonull.fit_transform(X_train) # escalar los datos de entrenamiento
X_test = scaler_nonull.transform(X_test) # transformar los datos de prueba

Ahora realizamos una primera evaluación de cada modelo y su rendimiento, para luego compararlos.

#### Regresion Lineal

In [117]:
model_lineal = LinearRegression() # definicion del modelo 

model_lineal.fit(X_train, y_train) # ajuste del modelo 

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_lineal = model_lineal.predict(X_train)
y_test_pred_lineal = model_lineal.predict(X_test)

# Calcular métricas en conjunto train
r2_train_lineal = np.round(r2_score(y_train, y_train_pred_lineal), 3)
rmse_train_lineal = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_lineal)), 0)
mae_train_lineal = np.round(mean_absolute_error(y_train, y_train_pred_lineal), 0)
mape_train_lineal = np.round(mean_absolute_percentage_error(y_train, y_train_pred_lineal), 2)

# Calcular métricas en conjunto test
r2_test_lineal = np.round(r2_score(y_test, y_test_pred_lineal), 3)
rmse_test_lineal = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_lineal)), 0)
mae_test_lineal = np.round(mean_absolute_error(y_test, y_test_pred_lineal), 0)
mape_test_lineal = np.round(mean_absolute_percentage_error(y_test, y_test_pred_lineal), 2)

# Mostrar métricas
print("R2 train:", r2_train_lineal)
print("RMSE - train:", rmse_train_lineal)
print("MAE - train:", mae_train_lineal)
print("MAPE - train:", mape_train_lineal)
print("")
print("R2 test:", r2_test_lineal)
print("RMSE - test:", rmse_test_lineal)
print("MAE - test:", mae_test_lineal)
print("MAPE - test:", mape_test_lineal)

R2 train: 0.445
RMSE - train: 4128.0
MAE - train: 1773.0
MAPE - train: 6.5

R2 test: 0.507
RMSE - test: 3853.0
MAE - test: 1818.0
MAPE - test: 4.65


In [123]:
# Observar Coeficientes de cada variable para cada modelo en un dataframe
coefficients_lineal = pd.DataFrame({'Variable':df_nonull.drop(["Immigrant count"], axis=1, inplace=False).columns})
coefficients_lineal['modelo_lineal']= model_lineal.coef_

# Mostrar coeficientes
coefficients_lineal.head(10)

,Variable,modelo_lineal
0,Year,-3551.677629
1,Unemployment %,4168.233046
2,Political and Violence Percentile,3093.667953
3,Probability of dying young,9362.225225
4,Rule of Law Percentile,-10403.449818
5,Salaried workers %,-2363.380181
6,GDP_growth,-1647.285606
7,Inflation_annual,1010.011995
8,Liberal democracy index,3987.376400
9,Health equality,550.863204


#### Regresion lineal - Huber (ventaja: bajo efecto de outliers)

In [124]:
model_huber = HuberRegressor(epsilon=1.15, alpha = 0.05) # definicion del modelo 

model_huber.fit(X_train, y_train) # ajuste del modelo 

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_huber = model_huber.predict(X_train)
y_test_pred_huber = model_huber.predict(X_test)

# Calcular métricas en conjunto train
r2_train_huber = np.round(r2_score(y_train, y_train_pred_huber), 3)
rmse_train_huber = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_huber)), 0)
mae_train_huber = np.round(mean_absolute_error(y_train, y_train_pred_huber), 0)
mape_train_huber = np.round(mean_absolute_percentage_error(y_train, y_train_pred_huber), 2)

# Calcular métricas en conjunto test
r2_test_huber = np.round(r2_score(y_test, y_test_pred_huber), 3)
rmse_test_huber = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_huber)), 0)
mae_test_huber = np.round(mean_absolute_error(y_test, y_test_pred_huber), 0)
mape_test_huber = np.round(mean_absolute_percentage_error(y_test, y_test_pred_huber), 2)

# Mostrar métricas
print("R2 train:", r2_train_huber)
print("RMSE - train:", rmse_train_huber)
print("MAE - train:", mae_train_huber)
print("MAPE - train:", mape_train_huber)
print("")
print("R2 test:", r2_test_huber)
print("RMSE - test:", rmse_test_huber)
print("MAE - test:", mae_test_huber)
print("MAPE - test:", mape_test_huber)

R2 train: 0.249
RMSE - train: 4800.0
MAE - train: 1306.0
MAPE - train: 1.39

R2 test: 0.277
RMSE - test: 4668.0
MAE - test: 1372.0
MAPE - test: 0.94


c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\linear_model\_huber.py:342: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [126]:
# Observar Coeficientes de cada variable para cada modelo en un dataframe
coefficients_huber = pd.DataFrame({'Variable':df_nonull.drop(["Immigrant count"], axis=1, inplace=False).columns})
coefficients_huber['modelo_lineal']= model_huber.coef_

# Mostrar coeficientes
coefficients_huber.head(10)

,Variable,modelo_lineal
0,Year,39.867744
1,Unemployment %,298.585706
2,Political and Violence Percentile,-119.777828
3,Probability of dying young,441.239407
4,Rule of Law Percentile,-141.951313
5,Salaried workers %,-495.752277
6,GDP_growth,-350.461215
7,Inflation_annual,520.854837
8,Liberal democracy index,304.233007
9,Health equality,-350.545236


#### Regresion lineal - RANSAC (ventaja: bueno para grandes outliers en "y")

In [127]:
model_ransac = RANSACRegressor(min_samples = 15) # definicion del modelo 

model_ransac.fit(X_train, y_train) # ajuste del modelo 

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_ransac = model_ransac.predict(X_train)
y_test_pred_ransac = model_ransac.predict(X_test)

# Calcular métricas en conjunto train
r2_train_ransac = np.round(r2_score(y_train, y_train_pred_ransac), 3)
rmse_train_ransac = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_ransac)), 0)
mae_train_ransac = np.round(mean_absolute_error(y_train, y_train_pred_ransac), 0)
mape_train_ransac = np.round(mean_absolute_percentage_error(y_train, y_train_pred_ransac), 2)

# Calcular métricas en conjunto test
r2_test_ransac = np.round(r2_score(y_test, y_test_pred_ransac), 3)
rmse_test_ransac = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_ransac)), 0)
mae_test_ransac = np.round(mean_absolute_error(y_test, y_test_pred_ransac), 0)
mape_test_ransac = np.round(mean_absolute_percentage_error(y_test, y_test_pred_ransac), 2)

# Mostrar métricas
print("R2 train:", r2_train_ransac)
print("RMSE - train:", rmse_train_ransac)
print("MAE - train:", mae_train_ransac)
print("MAPE - train:", mape_train_ransac)
print("")
print("R2 test:", r2_test_ransac)
print("RMSE - test:", rmse_test_ransac)
print("MAE - test:", mae_test_ransac)
print("MAPE - test:", mape_test_ransac)

R2 train: 0.033
RMSE - train: 5449.0
MAE - train: 1653.0
MAPE - train: 1.6

R2 test: 0.035
RMSE - test: 5394.0
MAE - test: 1745.0
MAPE - test: 0.87


#### Regresion lineal - TheilSen (ventaja: bueno para outliers pequeños tanto en "X" como en "y")

In [128]:
model_theilsen = TheilSenRegressor() # definicion del modelo 

model_theilsen.fit(X_train, y_train) # ajuste del modelo 

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_theilsen = model_theilsen.predict(X_train)
y_test_pred_theilsen = model_theilsen.predict(X_test)

# Calcular métricas en conjunto train
r2_train_theilsen = np.round(r2_score(y_train, y_train_pred_theilsen), 3)
rmse_train_theilsen = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_theilsen)), 0)
mae_train_theilsen = np.round(mean_absolute_error(y_train, y_train_pred_theilsen), 0)
mape_train_theilsen = np.round(mean_absolute_percentage_error(y_train, y_train_pred_theilsen), 2)

# Calcular métricas en conjunto test
r2_test_theilsen = np.round(r2_score(y_test, y_test_pred_theilsen), 3)
rmse_test_theilsen = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_theilsen)), 0)
mae_test_theilsen = np.round(mean_absolute_error(y_test, y_test_pred_theilsen), 0)
mape_test_theilsen = np.round(mean_absolute_percentage_error(y_test, y_test_pred_theilsen), 2)

# Mostrar métricas
print("R2 train:", r2_train_theilsen)
print("RMSE - train:", rmse_train_theilsen)
print("MAE - train:", mae_train_theilsen)
print("MAPE - train:", mape_train_theilsen)
print("")
print("R2 test:", r2_test_theilsen)
print("RMSE - test:", rmse_test_theilsen)
print("MAE - test:", mae_test_theilsen)
print("MAPE - test:", mape_test_theilsen)

R2 train: 0.372
RMSE - train: 4389.0
MAE - train: 1575.0
MAPE - train: 4.76

R2 test: 0.473
RMSE - test: 3985.0
MAE - test: 1557.0
MAPE - test: 3.4


#### Modelos lineales regularizados (Ridge, Lasso, E-Net)

**Buscar Alfa Optimo**

In [129]:
# Definir modelo Ridge y para Evaluar el valor del "alpha" óptimo
ridgecv = RidgeCV()
ridgecv.fit(X_train, y_train)
print("Alfa Optimo Ridge:", ridgecv.alpha_)

# Definir modelo Lasso y para Evaluar el valor del "alpha" óptimo
lassocv = LassoCV()
lassocv.fit(X_train, y_train)
print("Alfa Optimo Lasso:", lassocv.alpha_)

# Definir modelo E-Net y para Evaluar el valor del "alpha" óptimo
enetcv = ElasticNetCV()
enetcv.fit(X_train, y_train)
print("Alfa Optimo E-Net:", enetcv.alpha_)

Alfa Optimo Ridge: 1.0
Alfa Optimo Lasso: 0.92773669106235
Alfa Optimo E-Net: 1.7304206862222227


In [130]:
# Ingresamos el valor de alpha en una variable
alpha_opt_ridge = ridgecv.alpha_

# Ingresamos el valor de alpha en una variable
alpha_opt_lasso = lassocv.alpha_

# Ingresamos el valor de alpha en una variable
alpha_opt_enet = enetcv.alpha_

**Entrenar modelo con Alfa optimo**

In [131]:
# Definir modelo Ridge con nuestro valor optimo de alpha, entrenar y predecir
modelo_ridge = Ridge(alpha = alpha_opt_ridge)
y_test_ridge = modelo_ridge.fit(X_train, y_train).predict(X_test)

# Definir modelo Lasso con nuestro valor optimo de alpha, entrenar y predecir
modelo_lasso = Lasso(alpha = alpha_opt_lasso)
y_test_lasso = modelo_lasso.fit(X_train, y_train).predict(X_test)

# Definir modelo E-Net con nuestro valor optimo de alpha, entrenar y predecir
modelo_enet = ElasticNet(alpha = alpha_opt_enet)
y_test_enet = modelo_enet.fit(X_train, y_train).predict(X_test)

In [132]:
# Observar Coeficientes de cada variable para cada modelo en un dataframe
coefficients = pd.DataFrame({'Variable':df_nonull.drop(["Immigrant count"], axis=1, inplace=False).columns})
coefficients['modelo_ridge']= modelo_ridge.coef_
coefficients['modelo_lasso']= modelo_lasso.coef_
coefficients['modelo_net']= modelo_enet.coef_

# Mostrar coeficientes
coefficients

,Variable,modelo_ridge,modelo_lasso,modelo_net
0,Year,-3147.756559,-3099.376147,211.210996
1,Unemployment %,3380.453482,2904.497608,60.050911
2,Political and Violence Percentile,2538.179203,2426.272784,-56.472574
3,Probability of dying young,6928.625842,6581.308239,40.462542
4,Rule of Law Percentile,-9096.036039,-8292.332375,10.561849
...,...,...,...,...
64,Sub-region_Central America and Caribbean,89.346574,0.000000,-125.055870
65,Sub-region_European Union,366.765265,221.596086,16.642410
66,Sub-region_North America,761.530654,44.891180,-47.132018
67,Sub-region_Rest of Europe,-59.658470,-16.608889,-9.359894


**Evaluar Métricas**

In [137]:
# Métricas en test - Ridge
r2_test_ridge = np.round(r2_score(y_test, y_test_ridge), 3)
rmse_test_ridge = np.round(np.sqrt(mean_squared_error(y_test, y_test_ridge)), 0)
mae_test_ridge = np.round(mean_absolute_error(y_test, y_test_ridge), 0)
mape_test_ridge = np.round(mean_absolute_percentage_error(y_test, y_test_ridge), 2)

# Mostrar métricas - Ridge
print("R2 test - Ridge:", r2_test_ridge)
print("RMSE test - Ridge:", rmse_test_ridge)
print("MAE test - Ridge:", mae_test_ridge)
print("MAPE test - Ridge:", mape_test_ridge)

R2 test - Ridge: 0.507
RMSE test - Ridge: 3855.0
MAE test - Ridge: 1800.0
MAPE test - Ridge: 4.56


In [134]:
# Métricas en test - Lasso
r2_test_lasso = np.round(r2_score(y_test, y_test_lasso), 3)
rmse_test_lasso = np.round(np.sqrt(mean_squared_error(y_test, y_test_lasso)), 0)
mae_test_lasso = np.round(mean_absolute_error(y_test, y_test_lasso), 0)
mape_test_lasso = np.round(mean_absolute_percentage_error(y_test, y_test_lasso), 2)

# Mostrar métricas - Lasso
print("R2 test - Lasso:", r2_test_lasso)
print("RMSE test - Lasso:", rmse_test_lasso)
print("MAE test - Lasso:", mae_test_lasso)
print("MAPE test - Lasso:", mape_test_lasso)

R2 test - Lasso: 0.506
RMSE test - Lasso: 3859.0
MAE test - Lasso: 1797.0
MAPE test - Lasso: 4.52


In [135]:
# Métricas en test - E-Net
r2_test_enet = np.round(r2_score(y_test, y_test_enet), 3)
rmse_test_enet = np.round(np.sqrt(mean_squared_error(y_test, y_test_enet)), 0)
mae_test_enet = np.round(mean_absolute_error(y_test, y_test_enet), 0)
mape_test_enet = np.round(mean_absolute_percentage_error(y_test, y_test_enet), 2)

# Mostrar métricas - E-Net
print("R2 test - E-Net:", r2_test_enet)
print("RMSE test - E-Net:", rmse_test_enet)
print("MAE test - E-Net:", mae_test_enet)
print("MAPE test - E-Net:", mape_test_enet)

R2 test - E-Net: 0.103
RMSE test - E-Net: 5199.0
MAE test - E-Net: 2293.0
MAPE test - E-Net: 6.1


#### Desicion Tree

In [139]:
# Definir diccionario de valores para parámetros 
params = {'max_depth': range(6,8), 
          'min_samples_leaf' : [1, 3, 4], 
          'min_samples_split': [20, 30], 
          "criterion" : ["squared_error", "absolute_error", "poisson"] 
          } 

# Definir modelo y aplicar combinaciones de parametros según el diccinario 
tree = DecisionTreeRegressor() 
tree_cv = GridSearchCV(tree, params, cv = 3, refit = True, scoring = "neg_mean_squared_error")

# Entrenar modelo con cada combinación de parámetro 
tree_cv.fit(X_train, y_train) 

# Montrar los valores de los parámetros 
print(tree_cv.best_params_)

{'criterion': 'squared_error', 'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 20}


In [142]:
# Definir modelo con los mejores valores de parámetros 
tree_best =  DecisionTreeRegressor(max_depth = 8, 
                                   min_samples_leaf = 2,
                                   min_samples_split = 15, 
                                   criterion = tree_cv.best_params_['criterion']) 

# Entrenar con el conjunto de entrenamiento 
tree_best.fit(X_train, y_train) 

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_test_pred_tree = tree_best.predict(X_test) 
y_train_pred_tree = tree_best.predict(X_train) 

# Calcular métricas en conjunto train
r2_train_tree = np.round(r2_score(y_train, y_train_pred_tree), 3)
rmse_train_tree = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_tree)), 0)
mae_train_tree = np.round(mean_absolute_error(y_train, y_train_pred_tree), 0)
mape_train_tree = np.round(mean_absolute_percentage_error(y_train, y_train_pred_tree), 2)

# Calcular métricas en conjunto test
r2_test_tree = np.round(r2_score(y_test, y_test_pred_tree), 3)
rmse_test_tree = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_tree)), 0)
mae_test_tree = np.round(mean_absolute_error(y_test, y_test_pred_tree), 0)
mape_test_tree = np.round(mean_absolute_percentage_error(y_test, y_test_pred_tree), 2)

# Mostrar métricas
print("R2 - train:", r2_train_tree)
print("RMSE - train:", rmse_train_tree)
print("MAE - train:", mae_train_tree)
print("MAPE - train:", mape_train_tree)
print("")
print("R2 - test:", r2_test_tree)
print("RMSE - test:", rmse_test_tree)
print("MAE - test:", mae_test_tree)
print("MAPE - test:", mape_test_tree)


R2 - train: 0.838
RMSE - train: 2228.0
MAE - train: 903.0
MAPE - train: 1.84

R2 - test: 0.712
RMSE - test: 2947.0
MAE - test: 1125.0
MAPE - test: 1.46


#### Random Forest

In [37]:
# Definir diccionario de valores para parámetros 
params = {'n_estimators': [100], 
	      'criterion' : ['squared_error', 'friedman_mse', 'poisson'],
          "min_samples_split": [30, 50, 70], 
          'min_samples_leaf' : [2, 3, 5],
          "max_depth": [7, 8],
          }

# Definir modelo y aplicar combinaciones de parametros según el diccinario 
rf = RandomForestRegressor() 
rf_cv = GridSearchCV(rf, params, cv=3, scoring='neg_mean_squared_error').fit(X_train, y_train)

# Motrar mejores valores para parámeros
rf_cv.best_estimator_

RandomForestRegressor(criterion='poisson', max_depth=8, min_samples_leaf=2,
                      min_samples_split=30)

In [143]:
# Definir modelo con los mejores valores de parámetros (Nota: usar los mejores, pero hacer modificaciones para comparar metricas)
rf_best = RandomForestRegressor(n_estimators = 100, 
                           max_depth = 8, 
                           criterion = 'poisson', 
                           min_samples_split = 20, 
                           min_samples_leaf = 2,  
                           )

# Entrenar con el conjunto de entrenamiento 
rf_best.fit(X_train, y_train) 

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_rf = rf_best.predict(X_train)
y_test_pred_rf = rf_best.predict(X_test)

# Calcular métricas en conjunto train
r2_train_rf = np.round(r2_score(y_train, y_train_pred_rf), 3)
rmse_train_rf = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_rf)), 0)
mae_train_rf = np.round(mean_absolute_error(y_train, y_train_pred_rf), 0)
mape_train_rf = np.round(mean_absolute_percentage_error(y_train, y_train_pred_rf), 2)

# Calcular métricas en conjunto test
r2_test_rf = np.round(r2_score(y_test, y_test_pred_rf), 3)
rmse_test_rf = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_rf)), 0)
mae_test_rf = np.round(mean_absolute_error(y_test, y_test_pred_rf), 0)
mape_test_rf = np.round(mean_absolute_percentage_error(y_test, y_test_pred_rf), 2)

# Mostrar métricas
print("R2 - train:", r2_train_rf)
print("RMSE - train:", rmse_train_rf)
print("MAE - train:", mae_train_rf)
print("MAPE - train:", mape_train_rf)
print("")
print("R2 - test:", r2_test_rf)
print("RMSE - test:", rmse_test_rf)
print("MAE - test:", mae_test_rf)
print("MAPE - test:", mape_test_rf)

R2 - train: 0.801
RMSE - train: 2474.0
MAE - train: 752.0
MAPE - train: 0.85

R2 - test: 0.744
RMSE - test: 2775.0
MAE - test: 910.0
MAPE - test: 0.81


In [144]:
# Estimación de importancia relativa de variables en el modelo
imp_rel_rf = rf_best.feature_importances_
importancias = pd.DataFrame({"variable": X.columns, "importancia relativa": imp_rel_rf}) \
.sort_values(by='importancia relativa', ascending = False)
importancias[:15]

,variable,importancia relativa
57,Age group_All,0.467901
15,Number of residents,0.264668
0,Year,0.035849
52,Age group_25 - 34,0.029191
18,Number of Turist,0.022800
1,Unemployment %,0.022205
21,Año post_pandemia,0.017490
56,Age group_65+,0.017098
3,Probability of dying young,0.013276
55,Age group_55 - 64,0.012061


#### KNN

In [42]:
# Definir diccionario de valores para parámetros 
params = {'n_neighbors': range(1,20),
          'weights' : ['uniform', 'distance'],
          }

# Definir modelo y aplicar combinaciones de parametros según el diccinario entrenando el conjunto train
knn = KNeighborsRegressor()
knn_cv = GridSearchCV(knn, params, cv=3, scoring='neg_mean_squared_error').fit(X_train,y_train)

knn_cv.best_params_

{'n_neighbors': 11, 'weights': 'distance'}

In [145]:
knn_best =  KNeighborsRegressor(n_neighbors = 11, weights = 'uniform', leaf_size=30, p = 1)

knn_best.fit(X_train, y_train)

# Obtener predicciones con conjunto de entrenamiento y prueba
y_train_pred_knn = knn_best.predict(X_train)  
y_test_pred_knn = knn_best.predict(X_test)  

# Calcular métricas en conjunto train
r2_train_knn = np.round(r2_score(y_train, y_train_pred_knn), 3)
rmse_train_knn = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_knn)), 0)
mae_train_knn = np.round(mean_absolute_error(y_train, y_train_pred_knn), 0)
mape_train_knn = np.round(mean_absolute_percentage_error(y_train, y_train_pred_knn), 2)

# Calcular métricas en conjunto train
r2_test_knn = np.round(r2_score(y_test, y_test_pred_knn), 3)
rmse_test_knn = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_knn)), 0)
mae_test_knn = np.round(mean_absolute_error(y_test, y_test_pred_knn), 0)
mape_test_knn = np.round(mean_absolute_percentage_error(y_test, y_test_pred_knn), 2)

# Print metrics
print("R2 - train:", r2_train_knn)
print("RMSE - train:", rmse_train_knn)
print("MAE - train:", mae_train_knn)
print("MAPE - train:", mape_train_knn)
print("")
print("R2 - test:", r2_test_knn)
print("RMSE - test:", rmse_test_knn)
print("MAE - test:", mae_test_knn)
print("MAPE - test:", mape_test_knn)

R2 - train: 0.63
RMSE - train: 3369.0
MAE - train: 1019.0
MAPE - train: 1.21

R2 - test: 0.633
RMSE - test: 3325.0
MAE - test: 1136.0
MAPE - test: 1.52


#### SVR

In [5]:
# Definir diccionario de valores para parámetros 
params = {'kernel': ['rbf', 'linear'],
          'gamma': ['scale', 'auto'],
          'C' : [1.0, 0.85, 0.75] ,
          'max_iter': [-1, 100],
          "tol" : [0.001, 0.002, 0.0015, 0.1, 0.2]
          }

# Definir modelo y aplicar combinaciones de parametros según el diccinario 
svr = SVR()
svr_cv = GridSearchCV(svr, params, cv = 3, refit = True, scoring = 'neg_mean_squared_error') # elegir scoring deseano (r2, mae, mse, mape...)

# Entrenar modelo con cada combinación de parámetro 
svr_cv.fit(X_train, y_train)

# Motrar mejores valores para parámeros
svr_cv.best_params_

c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=

{'C': 1.0, 'gamma': 'scale', 'kernel': 'linear', 'max_iter': -1, 'tol': 0.0015}

In [146]:
# Definir modelo con mejors parámetros
svr_best = SVR(kernel = 'linear', 
               gamma = 'scale', 
               C = 1.0, 
               max_iter = -1, 
               tol = 0.0015
               )

# Enrenar con el conjunto de entrenamiento 
svr_best.fit(X_train, y_train) 

# Obtener predicciones con conjunto de entrenamiento y prueba
y_train_pred_svr = svr_best.predict(X_train)  
y_test_pred_svr = svr_best.predict(X_test)  

# Calcular métricas en conjunto train
r2_train_svr = np.round(r2_score(y_train, y_train_pred_svr), 3)
rmse_train_svr = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_svr)), 0)
mae_train_svr = np.round(mean_absolute_error(y_train, y_train_pred_svr), 0)
mape_train_svr = np.round(mean_absolute_percentage_error(y_train, y_train_pred_svr), 2)

# Calcular métricas en conjunto test
r2_test_svr = np.round(r2_score(y_test, y_test_pred_svr), 3)
rmse_test_svr = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_svr)), 0)
mae_test_svr = np.round(mean_absolute_error(y_test, y_test_pred_svr), 0)
mape_test_svr = np.round(mean_absolute_percentage_error(y_test, y_test_pred_svr), 2)

# Mostrar métricas
print("R2 - train:", r2_train_svr)
print("RMSE - train:", rmse_train_svr)
print("MAE - train:", mae_train_svr)
print("MAPE - train:", mape_train_svr)
print("")
print("R2 - test:", r2_test_svr)
print("RMSE - test:", rmse_test_svr)
print("MAE - test:", mae_test_svr)
print("MAPE - test:", mape_test_svr)


R2 - train: 0.026
RMSE - train: 5469.0
MAE - train: 1659.0
MAPE - train: 1.04

R2 - test: 0.022
RMSE - test: 5430.0
MAE - test: 1758.0
MAPE - test: 1.01


#### Red Neuronal Básica

Obtengamos primero un número de referencia en cuanto a el número de neuronas a usar. 

In [24]:
# Dos métodos para estimar numero de neuronas a usar
print(np.sqrt(len(X.columns)*2))
print(2/3 * len(X.columns) + 2)

11.575836902790225
46.666666666666664


In [60]:
# Definir diccionario de valores para parámetros 
params = {'max_iter': [200],
          'hidden_layer_sizes':[11, 44, (44, 11)],
          'batch_size': [30, 50, 70],
          'activation': ['relu', 'identity', 'tanh'],
          'alpha': [0.1, 0.01],
          'early_stopping' : [True],
          'solver' : ['adam', 'sgd', 'lbfgs']}

# Definir modelo y aplicar combinaciones de parametros según el diccinario 
rn = MLPRegressor()
rn_cv = GridSearchCV(rn, param_grid = params, cv = 3, scoring='neg_mean_squared_error', verbose=True, n_jobs = -1)

# Entrenar modelo con cada combinación de parámetro 
rn_cv.fit(X_train, y_train)

# Motrar mejores valores para parámeros
rn_cv.best_params_

Fitting 3 folds for each of 162 candidates, totalling 486 fits


c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
54 fits failed out of a total of 486.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
54 fits failed with the following error:
Traceback (most recent call last):
  File "c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 749, in fit
    return self._fit(X, y, incremental=False)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptr

{'activation': 'relu',
 'alpha': 0.01,
 'batch_size': 50,
 'early_stopping': True,
 'hidden_layer_sizes': 44,
 'max_iter': 200,
 'solver': 'lbfgs'}

In [153]:
# hacer un RN con los mejores parametros obtenidos y entrenar
rn_best = MLPRegressor(activation = 'relu',
                            alpha = 0.01, 
                           batch_size= 50, 
                            early_stopping = True, 
                           hidden_layer_sizes = 60, 
                            max_iter = 200, 
                           solver = 'lbfgs',
                            )

# Entrenar con el conjunto de entrenamiento 
rn_best.fit(X_train, y_train)

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_test_pred_rn = rn_best.predict(X_test) 
y_train_pred_rn = rn_best.predict(X_train) 

# Calculo de metricas en train
r2_train_rn = np.round(r2_score(y_train, y_train_pred_rn), 3)
rmse_train_rn = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_rn)), 0)
mae_train_rn = np.round(mean_absolute_error(y_train, y_train_pred_rn), 0)
mape_train_rn = np.round(mean_absolute_percentage_error(y_train, y_train_pred_rn), 2)

# Calculo de metricas en test
r2_test_rn = np.round(r2_score(y_test, y_test_pred_rn), 3)
rmse_test_rn = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_rn)), 0) 
mae_test_rn = np.round(mean_absolute_error(y_test, y_test_pred_rn), 0)
mape_test_rn = np.round(mean_absolute_percentage_error(y_test, y_test_pred_rn), 2)  

# Mostrar métricas
print("R2 - train:", r2_train_rn)
print("RMSE - train:", rmse_train_rn)
print("MAE - train:", mae_train_rn)
print("MAPE - train:", mape_train_rn)
print("")
print("R2 - test:", r2_test_rn)
print("RMSE - test:", rmse_test_rn)
print("MAE - test:", mae_test_rn)
print("MAPE - test:", mape_test_rn)

R2 - train: 0.976
RMSE - train: 866.0
MAE - train: 522.0
MAPE - train: 0.78

R2 - test: 0.94
RMSE - test: 1343.0
MAE - test: 629.0
MAPE - test: 0.62


### *Modelos Que Aceptan Datos Nulos*

Ahora evalueremos los datos con dos modelos que aceptan datos nulos (Hist Gradient Boosting y XGBoost), por lo que haremos una copia del conjunto de datos con todos los países y, al igual que antes, haremos de la variable "Year" una variable ordinal y el resto de variable categóricas a variables *dummy*.

In [154]:
# hacer copia del df_noagg
df_copy = df.copy()

# Transformar Year a variable ordinal de 1 (2008) a 15 (2022)
df_copy['Year'] = df_copy['Year'] - 2007

# Generar variables dummies a partir de nuestras variables categóricas "object" (no ordinales)
df_copy = pd.get_dummies(df_copy)

# Convertir las variables dummies booleanas en "int"
col_bool = df_copy.select_dtypes(include = ['bool']).columns
df_copy[col_bool] = df_copy[col_bool].astype(int)

# Verificar cambio
df_copy.info()
df_copy

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9360 entries, 0 to 9359
Data columns (total 71 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Year                                      9360 non-null   int64  
 1   Immigrant count                           9360 non-null   int64  
 2   Unemployment %                            9360 non-null   float64
 3   Political and Violence Percentile         9360 non-null   float64
 4   Probability of dying young                9360 non-null   float64
 5   Rule of Law Percentile                    9360 non-null   float64
 6   Salaried workers %                        9360 non-null   float64
 7   GDP_growth                                9360 non-null   float64
 8   Inflation_annual                          9360 non-null   float64
 9   Liberal democracy index                   9360 non-null   float64
 10  Health equality                     

,Year,Immigrant count,Unemployment %,Political and Violence Percentile,Probability of dying young,Rule of Law Percentile,Salaried workers %,GDP_growth,Inflation_annual,Liberal democracy index,...,Continent_America,Continent_Asia,Continent_Europe,Sub-region_Africa,Sub-region_Asia,Sub-region_Central America and Caribbean,Sub-region_European Union,Sub-region_North America,Sub-region_Rest of Europe,Sub-region_South America
0,1,759,11.33,14.90,3.7,24.52,67.41,2.40,15.31,0.164,...,0,0,0,1,0,0,0,0,0,0
1,1,2938,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
2,1,1128,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
3,1,265,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
4,1,156,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9355,15,330,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0
9356,15,146,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0
9357,15,169,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0
9358,15,99,5.60,6.60,5.8,25.00,42.14,4.71,13.96,0.234,...,0,1,0,0,1,0,0,0,0,0


Separemos el conjunto train/test y escalemos los datos.

In [155]:
# Separar variables input y variable target "Immigrant count" de df_copy
X = df_copy.drop("Immigrant count", axis = 1) # variables predictoras
y = df_copy["Immigrant count"]  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 58) # separar datos en conjunto train y test en un 75% / 25%
scaler_agg = MinMaxScaler() # definir scaler de datos 
X_train = scaler_agg.fit_transform(X_train) # escalar los datos de entrenamiento
X_test = scaler_agg.transform(X_test) # # escalar los datos de prueba

#### Hist Gradient Boosting

In [90]:
# Definir diccionario de valores para parámetros 
params = {'max_iter': [120], 
	      'loss' : ['squared_error', 'gamma', 'poisson'],
          "learning_rate": [0.1, 0.01], 
          'min_samples_leaf' : [2, 3, 5],
          "max_depth": [7, 8],
          'l2_regularization' : [0.0, 0.1, 0.3]
          }

# Definir modelo y aplicar combinaciones de parametros según el diccinario 
hgb = HistGradientBoostingRegressor() 
hgb_cv = GridSearchCV(hgb, params, cv=3, scoring='neg_mean_squared_error').fit(X_train, y_train)

hgb_cv.best_estimator_

c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
108 fits failed out of a total of 324.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
108 fits failed with the following error:
Traceback (most recent call last):
  File "c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\ensemble\_hist_gradient_boosting\gradient_boosting.py", line 353, in fit
    self._validate_params()
  File "c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "

HistGradientBoostingRegressor(l2_regularization=0.3, max_depth=8, max_iter=120,
                              min_samples_leaf=5)

In [156]:
# Definir modelo con los mejores valores de parámetros (Nota: usar los mejores, pero hacer modificaciones para comparar metricas)
hgb_best = HistGradientBoostingRegressor(
                           max_iter = 100, 
                           max_depth = 8,
                           loss = 'poisson', 
                           learning_rate = 0.08, 
                           min_samples_leaf = 36,
                           max_leaf_nodes = 30,
                           )

# Entrenar con el conjunto de entrenamiento 
hgb_best.fit(X_train, y_train) 

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_hgb = hgb_best.predict(X_train)
y_test_pred_hgb = hgb_best.predict(X_test)

# Calculo de metricas en train
r2_train_hgb = np.round(r2_score(y_train, y_train_pred_hgb), 3)
rmse_train_hgb = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_hgb)), 0)
mae_train_hgb = np.round(mean_absolute_error(y_train, y_train_pred_hgb), 0)
mape_train_hgb = np.round(mean_absolute_percentage_error(y_train, y_train_pred_hgb), 2)

# Calculo de metricas en train
r2_test_hgb = np.round(r2_score(y_test, y_test_pred_hgb), 3)
rmse_test_hgb = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_hgb)), 0)
mae_test_hgb = np.round(mean_absolute_error(y_test, y_test_pred_hgb), 0)
mape_test_hgb = np.round(mean_absolute_percentage_error(y_test, y_test_pred_hgb), 2)

# Mostrar métricas
print("R2 - train:", r2_train_hgb)
print("RMSE - train:", rmse_train_hgb)
print("MAE - train:", mae_train_hgb)
print("MAPE - train:", mape_train_hgb)
print('')
print("R2 - test:", r2_test_hgb)
print("RMSE - test:", rmse_test_hgb)
print("MAE - test:", mae_test_hgb)
print("MAPE - test:", mape_test_hgb)

R2 - train: 0.988
RMSE - train: 604.0
MAE - train: 294.0
MAPE - train: 0.41

R2 - test: 0.942
RMSE - test: 1319.0
MAE - test: 432.0
MAPE - test: 0.5


In [157]:
# Importancia por permutaciones
importancias_permu = permutation_importance(hgb_best, X_train, y_train, n_repeats=10, random_state=58)

# Importancias en dataframe
importances_hgb = pd.DataFrame({
    'feature': X.columns,
    'importance_mean': importancias_permu.importances_mean,
    'importance_std': importancias_permu.importances_std
}).sort_values(by='importance_mean', ascending=False)

importances_hgb.head(15)

,feature,importance_mean,importance_std
58,Age group_All,1.043920,0.064473
15,Number of residents,0.598518,0.067383
0,Year,0.182678,0.017346
18,Number of Turist,0.158322,0.024787
48,Sex_Both,0.086540,0.021279
1,Unemployment %,0.081296,0.009323
57,Age group_65+,0.058854,0.007528
5,Salaried workers %,0.044103,0.004677
56,Age group_55 - 64,0.043676,0.010874
9,Health equality,0.026059,0.004393


In [158]:
# Exportar tablas comparativas de metricas
importances_hgb.to_csv("../16 - Exports Modelos/agregados/feature_importance_hgb.csv", index = False)

#### XGBoost

In [5]:
# Definir diccionario de valores para parámetros 
params = {'objective' : ['reg:squarederror', 'reg:squaredlogerror']
          'eval_metric' : ['rmse'],   # rmsle disminuye efecto de outliers
          'booster' : ["gbtree", "gblinear", "dart"], # gbtree (default) y dart estan basados en arboles
          'n_estimators': [200],
          'max_depth': [7, 9], # dependiendo de la dimensionalidad de los datos, usar valores de profundidad menor
          "learning_rate" : [0.1, 0.3, 0.05]
          }

# Definir modelo y aplicar combinaciones de parametros según el diccinario 
xgb = XGBRegressor()
xgb_cv = GridSearchCV(xgb, params, cv=3, scoring = 'neg_mean_squared_error') # elegir scoring deseano (r2, mae, mse, mape...)

# Entrenar modelo con cada combinación de parámetro 
xgb_cv.fit(X_train,y_train)

# Motrar mejores valores para parámeros
xgb_cv.best_params_

C:\Users\Cristian De Andrade\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [08:29:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Cristian De Andrade\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [08:29:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Cristian De Andrade\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:160: UserWarning: [08:29:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "max_depth" } are not used.

  w

{'booster': 'dart',
 'eval_metric': 'rmse',
 'learning_rate': 0.3,
 'max_depth': 7,
 'n_estimators': 200,
 'objective': 'reg:squarederror'}

In [159]:
# Definir modelo con los mejores valores de parámetros (Nota: usar los mejores, pero hacer modificaciones al comparar metricas)
xgb_best = XGBRegressor(objective = 'reg:squarederror', 
                        eval_metric = 'rmse',
                        booster = 'dart',
                        n_estimators = 200,
                        max_depth = 7,
                        learning_rate = 0.3,
                        min_child_weight = 30
                    )

# Entrenar modelo con el conjunto de entrenamiento 
xgb_best.fit(X_train, y_train)

# Aplicar modelo sobre los datos de traint y test para predecir el target
y_train_pred_xgb = xgb_best.predict(X_train) 
y_test_pred_xgb = xgb_best.predict(X_test) 


# Calculate metrics for train set
r2_train_xgb = np.round(r2_score(y_train, y_train_pred_xgb), 3)
rmse_train_xgb = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred_xgb)), 0)
mae_train_xgb = np.round(mean_absolute_error(y_train, y_train_pred_xgb), 0)
mape_train_xgb = np.round(mean_absolute_percentage_error(y_train, y_train_pred_xgb), 2)

# Calculate metrics for test set
r2_test_xgb = np.round(r2_score(y_test, y_test_pred_xgb), 3)
rmse_test_xgb = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred_xgb)), 0)
mae_test_xgb = np.round(mean_absolute_error(y_test, y_test_pred_xgb), 0)
mape_test_xgb = np.round(mean_absolute_percentage_error(y_test, y_test_pred_xgb), 2)

# Print metrics
print("R2 - train:", r2_train_xgb)
print("RMSE - train:", rmse_train_xgb)
print("MAE - train:", mae_train_xgb)
print("MAPE - train:", mape_train_xgb)
print("")
print("R2 - test:", r2_test_xgb)
print("RMSE - test:", rmse_test_xgb)
print("MAE - test:", mae_test_xgb)
print("MAPE - test:", mape_test_xgb)


R2 - train: 0.983
RMSE - train: 707.0
MAE - train: 302.0
MAPE - train: 0.77

R2 - test: 0.884
RMSE - test: 1871.0
MAE - test: 572.0
MAPE - test: 0.99


### *Comparar Modelos*

Juntemos los resultados todos los modelos en un dataframe por tipo de métrica, ordenando por el mejor valor en el conjunto test.

In [160]:
# Juntar en un dataframe los datos
modelos_r2 = pd.DataFrame({
    'Modelo' : ['Lineal', 'Lineal - Huber', 'Lineal - RANSAC', 'Lineal - Theilsen', 'Lineal - Ridge', 'Lineal - Lasso', 'Lineal - E-Net', 'Decision Tree', 'Random Forest', 'KNN', 'SVR', 'Red Neuronal', 'HGB', 'XGBoost'],
    'R² train' : [r2_train_lineal, r2_train_huber, r2_train_ransac, r2_train_theilsen, np.nan, np.nan, np.nan, r2_train_tree, r2_train_rf, r2_train_knn, r2_train_svr, r2_train_rn, r2_train_hgb, r2_train_xgb],
    'R² test' : [r2_test_lineal, r2_test_huber, r2_test_ransac, r2_test_theilsen, r2_test_ridge, r2_test_lasso, r2_test_enet, r2_test_tree, r2_test_rf, r2_test_knn, r2_test_svr, r2_test_rn, r2_test_hgb, r2_test_xgb],
})

# Ordenar de forma descendente por R² test
modelos_r2.sort_values(by = 'R² test', ascending = False)

,Modelo,R² train,R² test
12,HGB,0.988,0.942
11,Red Neuronal,0.976,0.940
13,XGBoost,0.983,0.884
8,Random Forest,0.801,0.744
7,Decision Tree,0.838,0.712
9,KNN,0.630,0.633
0,Lineal,0.445,0.507
4,Lineal - Ridge,NaN,0.507
5,Lineal - Lasso,NaN,0.506
3,Lineal - Theilsen,0.372,0.473


In [164]:
# Juntar en un dataframe los datos
modelos_rsme = pd.DataFrame({
    'Modelo' : ['Lineal', 'Lineal - Huber', 'Lineal - RANSAC', 'Lineal - Theilsen', 'Lineal - Ridge', 'Lineal - Lasso', 'Lineal - E-Net', 'Decision Tree', 'Random Forest', 'KNN','SVR', 'Red Neuronal', 'HGB', 'XGBoost'],
    'RSME train' : [rmse_train_lineal, rmse_train_huber, rmse_train_ransac, rmse_train_theilsen, np.nan, np.nan, np.nan, rmse_train_tree, rmse_train_rf, rmse_train_knn, rmse_train_svr, rmse_train_rn, rmse_train_hgb, rmse_train_xgb],
    'RSME test' : [rmse_test_lineal, rmse_test_huber, rmse_test_ransac, rmse_test_theilsen, rmse_test_ridge, rmse_test_lasso, rmse_test_enet, rmse_test_tree, rmse_test_rf, rmse_test_knn, rmse_test_svr, rmse_test_rn, rmse_test_hgb, rmse_test_xgb],
    })

# Ordenar de forma ascendente por RSME test
modelos_rsme.sort_values(by = 'RSME test', ascending = True)

,Modelo,RSME train,RSME test
12,HGB,604.0,1319.0
11,Red Neuronal,866.0,1343.0
13,XGBoost,707.0,1871.0
8,Random Forest,2474.0,2775.0
7,Decision Tree,2228.0,2947.0
9,KNN,3369.0,3325.0
0,Lineal,4128.0,3853.0
4,Lineal - Ridge,NaN,3855.0
5,Lineal - Lasso,NaN,3859.0
3,Lineal - Theilsen,4389.0,3985.0


In [165]:
# Juntar en un dataframe los datos
modelos_mae = pd.DataFrame({
    'Modelo' : ['Lineal', 'Lineal - Huber', 'Lineal - RANSAC', 'Lineal - Theilsen', 'Lineal - Ridge', 'Lineal - Lasso', 'Lineal - E-Net', 'Decision Tree', 'Random Forest', 'KNN', 'SVR', 'Red Neuronal', 'HGB', 'XGBoost'],
    'MAE train' : [mae_train_lineal, mae_train_huber, mae_train_ransac, mae_train_theilsen, np.nan, np.nan, np.nan, mae_train_tree, mae_train_rf, mae_train_knn, mae_train_svr, mae_train_rn, mae_train_hgb, mae_train_xgb],
    'MAE test' : [mae_test_lineal, mae_test_huber, mae_test_ransac, mae_test_theilsen, mae_test_ridge, mae_test_lasso, mae_test_enet, mae_test_tree, mae_test_rf, mae_test_knn, mae_test_svr, mae_test_rn, mae_test_hgb, mae_test_xgb],
})

# Ordenar de forma descendente por MAE test
modelos_mae.sort_values(by = 'MAE test', ascending = True)

,Modelo,MAE train,MAE test
12,HGB,294.0,432.0
13,XGBoost,302.0,572.0
11,Red Neuronal,522.0,629.0
8,Random Forest,752.0,910.0
7,Decision Tree,903.0,1125.0
9,KNN,1019.0,1136.0
1,Lineal - Huber,1306.0,1372.0
3,Lineal - Theilsen,1575.0,1557.0
2,Lineal - RANSAC,1653.0,1745.0
10,SVR,1659.0,1758.0


In [163]:
# Juntar en un dataframe los datos
modelos_mape = pd.DataFrame({
    'Modelo' : ['Lineal', 'Lineal - Huber', 'Lineal - RANSAC', 'Lineal - Theilsen', 'Lineal - Ridge', 'Lineal - Lasso', 'Lineal - E-Net', 'Decision Tree', 'Random Forest', 'KNN', 'SVR', 'Red Neuronal', 'HGB', 'XGBoost'],
    'MAPE train' : [mape_train_lineal, mape_train_huber, mape_train_ransac, mape_train_theilsen, np.nan, np.nan, np.nan, mape_train_tree, mape_train_rf, mape_train_knn, mape_train_svr, mape_train_rn, mape_train_hgb, mape_train_xgb],
    'MAPE test' : [mape_test_lineal, mape_test_huber, mape_test_ransac, mape_test_theilsen, mape_test_ridge, mape_test_lasso, mape_test_enet, mape_test_tree, mape_test_rf, mape_test_knn, mape_test_svr, mape_test_rn, mape_test_hgb, mape_test_xgb],
})

# Ordenar de forma ascendente por MAPE test
modelos_mape.sort_values(by = 'MAPE test', ascending = True)

,Modelo,MAPE train,MAPE test
12,HGB,0.41,0.50
11,Red Neuronal,0.78,0.62
8,Random Forest,0.85,0.81
2,Lineal - RANSAC,1.60,0.87
1,Lineal - Huber,1.39,0.94
13,XGBoost,0.77,0.99
10,SVR,1.04,1.01
7,Decision Tree,1.84,1.46
9,KNN,1.21,1.52
3,Lineal - Theilsen,4.76,3.40


In [166]:
# Exportar tablas comparativas de metricas
modelos_r2.to_csv("../16 - Exports Modelos/agregados/metrics_r2_agg.csv", index = False)
modelos_rsme.to_csv("../16 - Exports Modelos/agregados/metrics_rsme_agg.csv", index = False)
modelos_mae.to_csv("../16 - Exports Modelos/agregados/metrics_mae_agg.csv", index = False)
modelos_mape.to_csv("../16 - Exports Modelos/agregados/metrics_mape_agg.csv", index = False)

De acuerdo a nuestra exploración inicial de modelos, HGB es el que da mejor resultados, seguido de la red neuronal (RN) de una capa. 

Estudiemos ahora el efecto de normalizar los datos sobre el modelo RN.

### *Transformar Datos para modelo de Redes Neuronales*

Considerando que en etapas previas observamos las variables inputs no presentan una distribución normal y que uno de los modelos con mejores métricas, además del HGB, fue el de RN de una capa, agregaremos unos pasos de transformación para normalizar los datos (target y target/inputs) para analizar una posible mejoría de las métricas en es

In [ ]:
# Funcion para calcular R² ajustado
def adjusted_r2(r2, n, p):
    return 1 - (1 - r2) * ((n - 1) / (n - p - 1))

**Normalizar (ajustar a Gaussiana) sólo la variable target ("Immigrant count")**

In [97]:
# hacer copia del df removiendo Senegal que presenta datos nulos para tasa de homicidios
df_nonull = df[df['Nationality code'] != 'SEN'].copy()

# Transformar Year a variable ordinal de 1 (2008) a 15 (2022)
df_nonull['Year'] = df_nonull['Year'] - 2007

# Generar variables dummies a partir de nuestras variables categóricas "object" (no ordinales)
df_nonull = pd.get_dummies(df_nonull)

# Convertir las variables dummies booleanas en "int"
col_bool = df_nonull.select_dtypes(include=['bool']).columns
df_nonull[col_bool] = df_nonull[col_bool].astype(int)

# Separar variables input y variable target "Immigrant count" de df_null (dataframe sin datos nulos)
X = df_nonull.drop("Immigrant count", axis=1)  # variables predictoras
y = df_nonull["Immigrant count"]  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=58)  # separar datos en conjunto train y test en un 75% / 25%
scaler_nonull = MinMaxScaler()  # definir scaler de datos
X_train = scaler_nonull.fit_transform(X_train)  # escalar los datos de entrenamiento
X_test = scaler_nonull.transform(X_test)  # transformar los datos de prueba

# Aplicar PowerTransformer a variable target
pt = PowerTransformer(method='box-cox')
y_train_trans = pt.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_trans = pt.transform(y_test.values.reshape(-1, 1)).flatten()

# Definir modelo con los mejores valores de parámetros (Nota: usar los mejores, pero hacer modificaciones para comparar metricas)
rn_best_trans1 = MLPRegressor(activation='relu',
                              alpha=0.01, 
                              batch_size=50, 
                              early_stopping=True, 
                              hidden_layer_sizes=(60), 
                              max_iter=200, 
                              solver='lbfgs')

# Entrenar con el conjunto de entrenamiento 
rn_best_trans1.fit(X_train, y_train_trans)

# Aplicar modelo sobre los datos de train y test para predecir el target 
y_train_pred_trans = rn_best_trans1.predict(X_train) 
y_test_pred_trans = rn_best_trans1.predict(X_test)

# Inverse transform the predictions
y_train_pred = pt.inverse_transform(y_train_pred_trans.reshape(-1, 1)).flatten()
y_test_pred = pt.inverse_transform(y_test_pred_trans.reshape(-1, 1)).flatten()

# Calculo de metricas en train 
r2_train_rn_trans1 = np.round(r2_score(y_train, y_train_pred), 3) 
rmse_train_rn_trans1 = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred)), 0) 
mae_train_rn_trans1 = np.round(mean_absolute_error(y_train, y_train_pred), 0) 
mape_train_rn_trans1 = np.round(mean_absolute_percentage_error(y_train, y_train_pred), 2)

# Calculo de metricas en test
r2_test_rn_trans1 = np.round(r2_score(y_test, y_test_pred), 3) 
rmse_test_rn_trans1 = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred)), 0) 
mae_test_rn_trans1 = np.round(mean_absolute_error(y_test, y_test_pred), 0) 
mape_test_rn_trans1 = np.round(mean_absolute_percentage_error(y_test, y_test_pred), 2)

# Calcular R² Ajustado
adj_r2_train_rn_trans1 = adjusted_r2(r2_train_rn_trans1, len(y_train), X_train.shape[1])
adj_r2_test_rn_trans1 = adjusted_r2(r2_test_rn_trans1, len(y_test), X_test.shape[1])

# Mostrar métricas
print("R2 - train:", r2_train_rn_trans1)
print("Adjusted R2 - train:", np.round(adj_r2_train_rn_trans1, 3))
print("RMSE - train:", rmse_train_rn_trans1)
print("MAE - train:", mae_train_rn_trans1)
print("MAPE - train:", mape_train_rn_trans1)
print('')
print("R2 - test:", r2_test_rn_trans1)
print("Adjusted R2 - train:", np.round(adj_r2_test_rn_trans1, 3))
print("RMSE - test:", rmse_test_rn_trans1)
print("MAE - test:", mae_test_rn_trans1)
print("MAPE - test:", mape_test_rn_trans1)

R2 - train: 0.975
Adjusted R2 - train: 0.975
RMSE - train: 875.0
MAE - train: 284.0
MAPE - train: 0.14

R2 - test: 0.965
Adjusted R2 - train: 0.964
RMSE - test: 1023.0
MAE - test: 359.0
MAPE - test: 0.17


**Normalizar (ajustar a Gaussiana) variables inputs y variable target (sin scaler)**

In [114]:
# hacer copia del df removiendo Senegal que presenta datos nulos para tasa de homicidios
df_nonull = df[df['Nationality code'] != 'SEN'].copy()

# Transformar Year a variable ordinal de 1 (2008) a 15 (2022)
df_nonull['Year'] = df_nonull['Year'] - 2007

# Generar variables dummies a partir de nuestras variables categóricas "object" (no ordinales)
df_nonull = pd.get_dummies(df_nonull)

# Convertir las variables dummies booleanas en "int"
col_bool = df_nonull.select_dtypes(include=['bool']).columns
df_nonull[col_bool] = df_nonull[col_bool].astype(int)

# Separar variables input y variable target "Immigrant count" de df_null (dataframe sin datos nulos)
X = df_nonull.drop("Immigrant count", axis=1)  # variables predictoras
y = df_nonull["Immigrant count"]  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=58)  # separar datos en conjunto train y test en un 75% / 25%

# Aplicar PowerTransformer a variables input
pt_X = PowerTransformer(method='yeo-johnson')
X_train_trans = pt_X.fit_transform(X_train)
X_test_trans = pt_X.transform(X_test)

# Aplicar PowerTransformer a variable target
pt_y = PowerTransformer(method='box-cox')
y_train_trans = pt_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_trans = pt_y.transform(y_test.values.reshape(-1, 1)).flatten()

# Definir modelo con los mejores valores de parámetros
rn_best_trans2 = MLPRegressor(activation='relu',
                              alpha=0.01, 
                              batch_size=50, 
                              early_stopping=True, 
                              hidden_layer_sizes=(60), 
                              max_iter=200, 
                              solver='lbfgs')

# # Entrenar con el conjunto de entrenamiento 
rn_best_trans2.fit(X_train_trans, y_train_trans)

# Aplicar modelo sobre los datos de train y test para predecir el target 
y_train_pred_trans = rn_best_trans2.predict(X_train_trans) 
y_test_pred_trans = rn_best_trans2.predict(X_test_trans)

# Transformacion inversa de predicciones
y_train_pred = pt_y.inverse_transform(y_train_pred_trans.reshape(-1, 1)).flatten()
y_test_pred = pt_y.inverse_transform(y_test_pred_trans.reshape(-1, 1)).flatten()

# Calculo de metricas en train 
r2_train_rn_trans2 = np.round(r2_score(y_train, y_train_pred), 3) 
rmse_train_rn_trans2 = np.round(np.sqrt(mean_squared_error(y_train, y_train_pred)), 0) 
mae_train_rn_trans2 = np.round(mean_absolute_error(y_train, y_train_pred), 0) 
mape_train_rn_trans2 = np.round(mean_absolute_percentage_error(y_train, y_train_pred), 2)

# Calculo de metricas en test
r2_test_rn_trans2 = np.round(r2_score(y_test, y_test_pred), 3) 
rmse_test_rn_trans2 = np.round(np.sqrt(mean_squared_error(y_test, y_test_pred)), 0) 
mae_test_rn_trans2 = np.round(mean_absolute_error(y_test, y_test_pred), 0) 
mape_test_rn_trans2 = np.round(mean_absolute_percentage_error(y_test, y_test_pred), 2)

# Calcular Adjusted R²
adj_r2_train_rn_trans2 = adjusted_r2(r2_train_rn, len(y_train), X_train.shape[1])
adj_r2_test_rn_trans2 = adjusted_r2(r2_test_rn, len(y_test), X_test.shape[1])

# Mostrar métricas
print("R2 - train:", r2_train_rn_trans2)
print("Adjusted R2 - train:", np.round(adj_r2_train_rn_trans2, 3))
print("RMSE - train:", rmse_train_rn_trans2)
print("MAE - train:", mae_train_rn_trans2)
print("MAPE - train:", mape_train_rn_trans2)
print('')
print("R2 - test:", r2_test_rn_trans2)
print("Adjusted R2 - test:", np.round(adj_r2_test_rn_trans2, 3))
print("RMSE - test:", rmse_test_rn_trans2)
print("MAE - test:", mae_test_rn_trans2)
print("MAPE - test:", mape_test_rn_trans2)

R2 - train: 0.988
Adjusted R2 - train: 0.988
RMSE - train: 613.0
MAE - train: 211.0
MAPE - train: 0.11

R2 - test: 0.976
Adjusted R2 - test: 0.975
RMSE - test: 853.0
MAE - test: 293.0
MAPE - test: 0.14


In [108]:
# Juntar metricas en un dataframe los datos
modelos_rn_trans = pd.DataFrame({
    'Métricas' : ['R² test', 'R² adjusted test', 'RMSE test', 'MAE test', 'MAPE test'],
    'RN + target normalizado' : [r2_test_rn_trans1, np.round(adj_r2_test_rn_trans1, 3), rmse_test_rn_trans1, mae_test_rn_trans1, mape_test_rn_trans1],
    'RN + inputs/target normalizado' : [r2_test_rn_trans2, np.round(adj_r2_test_rn_trans2, 3), rmse_test_rn_trans2, mae_test_rn_trans2, mape_test_rn_trans2],
})

modelos_rn_trans

,Métricas,RN + target normalizado,RN + inputs/target normalizado
0,R² test,0.965,0.976
1,R² adjusted test,0.964,0.975
2,RMSE test,1023.000,853.000
3,MAE test,359.000,293.000
4,MAPE test,0.170,0.140


In [109]:
# Exportar tablas comparativas de metricas
modelos_rn_trans.to_csv("../16 - Exports Modelos/agregados/rn_trans_metrics.csv", index = False)

Vemos que la normalización de inputs y target mejoró considerablemente los resultados del modelo de red neuronal de una capa, siendo ahora el modelo con mejor rendiemiento de todos los evaluados hasta el momento. 

Seleccinaremos este modelo como el mejor para el conjunto de datos con agregados y pasaremos a realizar prediciones a futuro para compararlas luego con el modelo sin agregados.

In [167]:
# Guardar mejor modelo
with open('../16 - Exports Modelos/agregados/rn_trans_agg.pkl', 'wb') as file:
    pickle.dump(rn_best_trans2, file)

# Guardar PowerTransformer de inputs
joblib.dump(pt_X, '../16 - Exports Modelos/agregados/power_transformer_X.pkl')

# Guardar PowerTransformerpara target
joblib.dump(pt_y, '../16 - Exports Modelos/agregados/power_transformer_y.pkl')

['../16 - Exports Modelos/agregados/power_transformer_y.pkl']

### *Predicciones con el mejor modelo seleccionado*

Ahora obtengamos las predicicones para todo el conjunto de datos incluyendo dos años adicionales (2023 y 2024) para dos nacionalidades con imigracion alta y media: Colombia y Brasil. Nuetsro onjetivo es comparar los resultados de este modelo con agregadas "Both" y "All" de sexo y grupo de edad, frente al mejor modelo obtenido sin estos agregados.

*Nota: Los datos de los años 2023 y 2024 fueron preparados investigando los valores de variables inputs o planteando escenarios con base a la tendencia de los valores en años previos.*

**Importar los datos acomodados**

In [3]:
# Importar el dataset
df_predicciones = pd.read_csv("../17 - Prediciones/datos a predecir.csv")

df_predicciones.info()
df_predicciones

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9456 entries, 0 to 9455
Data columns (total 28 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Year                               9456 non-null   int64  
 1   Nationality code                   9456 non-null   object 
 2   Sex                                9456 non-null   object 
 3   Age group                          9456 non-null   object 
 4   Immigrant count                    9360 non-null   float64
 5   Unemployment %                     9456 non-null   float64
 6   Political and Violence Percentile  9456 non-null   float64
 7   Probability of dying young         9456 non-null   float64
 8   Rule of Law Percentile             9456 non-null   float64
 9   Salaried workers %                 9456 non-null   float64
 10  GDP_growth                         9456 non-null   float64
 11  Inflation_annual                   9456 non-null   float

,Year,Nationality code,Sex,Age group,Immigrant count,Unemployment %,Political and Violence Percentile,Probability of dying young,Rule of Law Percentile,Salaried workers %,...,Non-state_deaths,Intrastate_deaths,Interstate_deaths,Number of residents,Political regime,Homicide Rate,Number of Turist,Spanish language,Restricciones_pandemia,Año post_pandemia
0,2008,DZA,Both,0 - 14,759.0,11.33,14.90,3.7,24.52,67.41,...,0,345,0,51922,3,0.95,44400000,0,0,0
1,2008,PER,Males,35 - 44,2938.0,4.03,17.31,5.1,25.96,44.47,...,0,40,0,60185,7,5.27,44400000,1,0,0
2,2008,PER,Males,45 - 54,1128.0,4.03,17.31,5.1,25.96,44.47,...,0,40,0,60185,7,5.27,44400000,1,0,0
3,2008,PER,Males,55 - 64,265.0,4.03,17.31,5.1,25.96,44.47,...,0,40,0,60185,7,5.27,44400000,1,0,0
4,2008,PER,Males,65+,156.0,4.03,17.31,5.1,25.96,44.47,...,0,40,0,60185,7,5.27,44400000,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9451,2024,BRA,Males,35 - 44,NaN,7.00,34.00,7.7,45.40,68.04,...,2100,0,0,92400,6,21.40,70500000,0,0,0
9452,2024,BRA,Males,45 - 54,NaN,7.00,34.00,7.7,45.40,68.04,...,2100,0,0,92400,6,21.40,70500000,0,0,0
9453,2024,BRA,Males,55 - 64,NaN,7.00,34.00,7.7,45.40,68.04,...,2100,0,0,92400,6,21.40,70500000,0,0,0
9454,2024,BRA,Males,65+,NaN,7.00,34.00,7.7,45.40,68.04,...,2100,0,0,92400,6,21.40,70500000,0,0,0


In [4]:
# hacer copia del df removiendo Senegal que presenta datos nulos para tasa de homicidios
df_nonull = df_predicciones[df_predicciones['Nationality code'] != 'SEN']
df_nonull_copy = df_nonull.copy()

# Transformar Year a variable ordinal de 1 (2008) a 15 (2022)
df_nonull_copy['Year'] = df_nonull_copy['Year'] - 2007

# Generar variables dummies a partir de nuestras variables categóricas "object" (no ordinales)
df_nonull_copy = pd.get_dummies(df_nonull_copy)

# Convertir las variables dummies booleanas en "int"
col_bool = df_nonull_copy.select_dtypes(include = ['bool']).columns
df_nonull_copy[col_bool] = df_nonull_copy[col_bool].astype(int)

# Verificar cambio
df_nonull_copy.info()
df_nonull_copy

<class 'pandas.core.frame.DataFrame'>
Index: 9096 entries, 0 to 9455
Data columns (total 70 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Year                                      9096 non-null   int64  
 1   Immigrant count                           9000 non-null   float64
 2   Unemployment %                            9096 non-null   float64
 3   Political and Violence Percentile         9096 non-null   float64
 4   Probability of dying young                9096 non-null   float64
 5   Rule of Law Percentile                    9096 non-null   float64
 6   Salaried workers %                        9096 non-null   float64
 7   GDP_growth                                9096 non-null   float64
 8   Inflation_annual                          9096 non-null   float64
 9   Liberal democracy index                   9096 non-null   float64
 10  Health equality                          

,Year,Immigrant count,Unemployment %,Political and Violence Percentile,Probability of dying young,Rule of Law Percentile,Salaried workers %,GDP_growth,Inflation_annual,Liberal democracy index,...,Continent_America,Continent_Asia,Continent_Europe,Sub-region_Africa,Sub-region_Asia,Sub-region_Central America and Caribbean,Sub-region_European Union,Sub-region_North America,Sub-region_Rest of Europe,Sub-region_South America
0,1,759.0,11.33,14.90,3.7,24.52,67.41,2.40,15.31,0.164,...,0,0,0,1,0,0,0,0,0,0
1,1,2938.0,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
2,1,1128.0,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
3,1,265.0,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
4,1,156.0,4.03,17.31,5.1,25.96,44.47,9.13,1.10,0.649,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9451,17,NaN,7.00,34.00,7.7,45.40,68.04,2.01,4.00,0.670,...,1,0,0,0,0,0,0,0,0,1
9452,17,NaN,7.00,34.00,7.7,45.40,68.04,2.01,4.00,0.670,...,1,0,0,0,0,0,0,0,0,1
9453,17,NaN,7.00,34.00,7.7,45.40,68.04,2.01,4.00,0.670,...,1,0,0,0,0,0,0,0,0,1
9454,17,NaN,7.00,34.00,7.7,45.40,68.04,2.01,4.00,0.670,...,1,0,0,0,0,0,0,0,0,1


In [5]:
# Separar variables input y variable target "Immigrant count" de df_null (dataframe sin atos nulos)
X = df_nonull_copy.drop("Immigrant count", axis = 1) # variables predictoras
y = df_nonull_copy["Immigrant count"]  # Target

In [6]:
# Cagar PowerTransformers y modelo
pt_X = joblib.load('../16 - Exports Modelos/agregados/power_transformer_X.pkl')
pt_y = joblib.load('../16 - Exports Modelos/agregados/power_transformer_y.pkl')
modelo_final = joblib.load('../16 - Exports Modelos/agregados/rn_trans_agg.pkl')

# Transformar variables inputs
X_trans = pt_X.transform(X)

# Predecir
y_pred_trans = modelo_final.predict(X_trans)

# Transformación inversa de prediccicones para obtener escala normal
y_pred = pt_y.inverse_transform(y_pred_trans.reshape(-1, 1)).flatten()

**Intervalos de confianza**

Adicionalmente, estimemos un intervalo de confianza del 90% para nuestras predicciones en base a nuestro modelo elegido. Para ello usaremos el método de Conformal Prediction.

In [7]:
# Hacer copia del df removiendo Senegal que presenta datos nulos para tasa de homicidios
df_nonull = df_predicciones[df_predicciones['Nationality code'] != 'SEN']
df_nonull_copy = df_nonull.copy()

# Remover años 2023 y 2024 de target e inputs que poseen datos nulos (se incluyen para prediciones futuras)
df_nonull_copy = df_nonull_copy[(df_nonull_copy['Year'] != 2023) & (df_nonull_copy['Year'] != 2024)]

# Transformar Year a variable ordinal de 1 (2008) a 15 (2022)
df_nonull_copy['Year'] = df_nonull_copy['Year'] - 2007

# Generar variables dummies a partir de nuestras variables categóricas "object" (no ordinales)
df_nonull_copy = pd.get_dummies(df_nonull_copy)

# Convertir las variables dummies booleanas en "int"
col_bool = df_nonull_copy.select_dtypes(include = ['bool']).columns
df_nonull_copy[col_bool] = df_nonull_copy[col_bool].astype(int)

# Separar target de inputs
X = df_nonull_copy.drop("Immigrant count", axis = 1) # variables predictoras
y = df_nonull_copy["Immigrant count"]  # Target

# Separar entrenamiento y prueba para el bootstrap
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=58)  # separar datos en conjunto train y test en un 75% / 25%

# Aplicar PowerTransformer a variables input y target de entrenamiento
X_train_trans = pt_X.fit_transform(X_train)
y_train_trans = pt_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()

# Definir modelo con los mejores valores de parámetros (Nota: usar los mejores, pero hacer modificaciones para comparar metricas)
rn_trans = MLPRegressor(activation='relu',
                              alpha=0.01, 
                              batch_size=50, 
                              early_stopping=True, 
                              hidden_layer_sizes=(60), 
                              max_iter=200, 
                              solver='lbfgs')


# MapieRegressor
mapie = MapieRegressor(rn_trans, method="naive")

# Fit MapieRegressor en datos de entrenamiento
mapie.fit(X_train_trans, y_train_trans)

# Predecir intervalos en datos
predictions = mapie.predict(X_trans, alpha=0.1)  # 90% intervalo de confianza

# Obtener la tupla correspondiente a los intervalos
y_test_pred_interval = predictions[1]

# Extraer valores de intervalos
y_pred_interval_lower_t = y_test_pred_interval[:, 0]
y_pred_interval_upper_t = y_test_pred_interval[:, 1]

# Transform inverse predictions and intervals
y_pred_lower = pt_y.inverse_transform(y_pred_interval_lower_t.reshape(-1, 1)).flatten()
y_pred_upper = pt_y.inverse_transform(y_pred_interval_upper_t.reshape(-1, 1)).flatten()

c:\ProgramFiles\Anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


**Agregar predicicones e intervalos de conf. al dataframe y exportar**

In [8]:
# Hacer copia de dataframe inicial sin nulos y variables inputs que no necesitamos
df_pred = df_nonull.iloc[:, :5].copy()

# Agregar predicicones al dataframe
df_pred['Predictions_rnn_agg'] = np.round(y_pred, 0).astype(int)
df_pred['Immigrant count'] = df_pred['Immigrant count'].astype('Int64')
df_pred['Lower limit_agg'] = np.round(y_pred_lower, 0).astype(int)
df_pred['Upper limit_agg'] = np.round(y_pred_upper, 0).astype(int)

df_pred.info()
df_pred

<class 'pandas.core.frame.DataFrame'>
Index: 9096 entries, 0 to 9455
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Year                 9096 non-null   int64 
 1   Nationality code     9096 non-null   object
 2   Sex                  9096 non-null   object
 3   Age group            9096 non-null   object
 4   Immigrant count      9000 non-null   Int64 
 5   Predictions_rnn_agg  9096 non-null   int32 
 6   Lower limit_agg      9096 non-null   int32 
 7   Upper limit_agg      9096 non-null   int32 
dtypes: Int64(1), int32(3), int64(1), object(3)
memory usage: 541.9+ KB


,Year,Nationality code,Sex,Age group,Immigrant count,Predictions_rnn_agg,Lower limit_agg,Upper limit_agg
0,2008,DZA,Both,0 - 14,759,761,709,1111
1,2008,PER,Males,35 - 44,2938,1693,1572,2422
2,2008,PER,Males,45 - 54,1128,769,576,907
3,2008,PER,Males,55 - 64,265,206,174,281
4,2008,PER,Males,65+,156,164,138,224
...,...,...,...,...,...,...,...,...
9451,2024,BRA,Males,35 - 44,<NA>,2391,1457,2250
9452,2024,BRA,Males,45 - 54,<NA>,1496,724,1134
9453,2024,BRA,Males,55 - 64,<NA>,720,357,568
9454,2024,BRA,Males,65+,<NA>,311,331,528


In [9]:
# Exportar prediccicones de modelo seleccionado para el conjunto de datos con agregados
df_pred.to_csv("../17 - Prediciones/predicciones_rn_agg.csv", index = False)

En el siguiente link se puede ver la comparativa de las prediciones de los modelos con y sin agregados: https://public.tableau.com/app/profile/cristian.de.andrade.correia/viz/PrediccindeInmigrantesenEspaa/Dashboard1